In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
from numba import njit
import numpy as np
import pandas as pd
import pdr
from pyarrow import parquet as pq
from rich import print
from scipy.optimize import curve_fit
from scipy.stats import halfnorm

from glcat_photometry_utils import (
    mcmc_aperture_curve, 
    plot_mcmc_results, 
    get_percentile_ranges, 
    plot_mcmc_walkers,
    gaussian_flux_model
)

%matplotlib inline

np.random.seed(555)

In [ ]:
GPHOTON_ROOT = Path("/home/michael/Desktop/gPhoton2")

metadata = pq.read_table(GPHOTON_ROOT / "gPhoton/aspect/metadata.parquet").to_pandas()
ncat = pq.read_table('../e23456_ncat.parquet')

In [ ]:
ffull = pdr.read(GPHOTON_ROOT / "test_data/e23456/e23456-nd-ffull-b00-image-g.fits.gz")
expt = pd.read_csv(
    GPHOTON_ROOT / "test_data/e23456/e23456-nd-f0120-b00-movie-exptime.csv", 
    index_col=None
)
expt = expt['expt'].sum()  # not sure this is right?

In [ ]:
# same function, different signature -- curve_fit requires the dependent 
# variable(s) to go first.

@njit(cache=True)
def gaussian_flux_model_curvefit(r, total_flux, sigma, background):
    return (
        total_flux 
        * (1 - np.exp(-r**2 / (2 * sigma**2))) 
        + background * np.pi * r**2
    )

In [ ]:
source_ix=400
aperture_radii = np.array([1.5, 2.3, 3.8, 6.0, 9.0])#, 12.8, 17.3])
flux = np.array([ncat.column(f'NUV_CPS_APER{a}')[source_ix].as_py() for a, r in enumerate(aperture_radii)])
flux_err = np.array([ncat.column(f'NUV_CPS_ERR_APER{a}')[source_ix].as_py() for a,r in enumerate(aperture_radii)])
ix = np.where(np.isfinite(flux))

In [ ]:
flat_samples,samples = mcmc_aperture_curve(aperture_radii,flux,flux_err,
                        nsteps = 1000, # number of MCMC steps
                        burnin = 200, # number of burn-in steps
                        nwalkers = 32, # number of mc walkers
                       )

percentiles=[16,50,84]
labels = ["cps", "sigma", "bg_cps"]

# cut the 'continuous' extension here because i wanted to make it 
# easier to eyeball outputs of the two algorithms.
# there is of course no reason you couldn't add it back for both.

model_samples = np.zeros((len(flat_samples), len(aperture_radii)))
for i, (tf, sig, back) in enumerate(flat_samples):
    model_samples[i] = gaussian_flux_model(tf, sig, back, aperture_radii)

# Calculate the median and confidence intervals of the model predictions
model_percentiles = np.percentile(model_samples, [16, 50, 84], axis=0)
mc_params = [np.median(flat_samples[:, i]) for i in range(3)]
# yes, I know this simplifies
mc_sigmas = (
    (model_percentiles[0] - model_percentiles[1]) / 2
    + (model_percentiles[2] - model_percentiles[1]) / 2
)

In [ ]:
guess = [
    np.mean(flux), 
    5/2.355, 
    (flux[-1]-flux[-2])/(np.pi * (aperture_radii[-1]**2-aperture_radii[-2]**2))
]
# min, then max, for total flux, sigma, background
bounds = ((0, 0, 0), (1000, 10, 100))

# this don't have to be kwargs, just making it explicit
cf_params, cf_cov = curve_fit(
    f=gaussian_flux_model_curvefit,
    xdata=aperture_radii,
    ydata=flux,
    p0=guess,
    sigma=flux_err,
    absolute_sigma=True,
    check_finite=True,
    # dogbox turns out to be unstable on these data.
    method='trf',
    bounds=bounds,
)
# 1-sigma errors for each model parameter
cf_param_errs = np.sqrt(np.diag(cf_cov))

# crude sampling estimation of estimate error. this may not be 
# the best way to do it.
dists = [
    halfnorm.rvs(p, s, 3200) for (p, s) in zip(cf_params, cf_param_errs)
]
cf_sigmas = [
    np.std(gaussian_flux_model_curvefit(r, *dists)) 
    for r in aperture_radii
]
print(f"cf sigmas: {cf_sigmas}")
print(f"mcmc sigmas: {mc_sigmas}")

In [ ]:
cf_residuals = gaussian_flux_model_curvefit(aperture_radii, *cf_params) - flux
mcmc_residuals = gaussian_flux_model(*mc_params, aperture_radii) - flux

# supplementary goodness-of-fit metric of your choice goes here

In [ ]:
print(mc_params)
print(cf_params)
print(abs(mc_params - cf_params) / mc_params)